In [1]:
from matplotlib import pyplot as plt
import time
import datetime
from utils_laj import *
from data_processing import get_SensorData, my_pca, series_to_supervised, lstm_sampling
from sklearn.model_selection import train_test_split
import pandas as pd

today = datetime.date.today()
TR_END = "2017-12-31 23:00:00"

In [2]:
path_checkpoint = './save/save_lstm/lstm_2_layers'
input_path = './input_data'
file = os.path.join(input_path,'sensors.csv')
target_file = os.path.join(input_path,'coke_target.csv')

In [3]:
# Считываем данные из файлов, нормализуем и применяем метод главных комполнентов
X, y, submit_X, mean_y, std_y = get_SensorData(file, target_file, nc=12)


feature shape: (13272, 48)


/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


target length: (10400, 2)
Explained variance [0.50143148 0.12808438 0.06505652 0.04321608 0.03529743 0.0335428
 0.02754238 0.0183814  0.01835391 0.0181078  0.01717936 0.0154992 ], Cumsum [0.50143148 0.62951587 0.69457238 0.73778846 0.77308589 0.80662869
 0.83417107 0.85255247 0.87090638 0.88901418 0.90619354 0.92169274]


In [4]:
# Размерности датасета с известным целевым y и полного датасета
print("Data X: %s, y %s, submit_X %s" % (X.shape, y.shape, submit_X.shape))

Data X: (10400, 12), y (10400, 2), submit_X (13272, 12)


In [5]:
# выделим последние записи в размере контрольного датасета в тестовый датасет
x1_train, x1_test, y1_train, y1_test = train_test_split(X, y, test_size=2920, random_state=42, shuffle=False)

# Формируем сэмплы для LSTM по 48 рядов внахлест с шагом 1
timesteps = 48
lag = 1
i_train = len(x1_train) + timesteps  


x_train, y_train = lstm_sampling(X[:i_train], y=y[:i_train]['target'], timesteps=timesteps, lag=lag)
x_test, y_test = lstm_sampling(x1_test, y=y1_test['target'], timesteps=timesteps, lag=lag)
#yl = y['target'][timesteps:]

# формируем данные сабмита по аналогичному принципу + берем 47 шагов из тестовых данных
Xs, ys = lstm_sampling(submit_X, timesteps=timesteps, lag=lag)

print("Data x_train, y_train: (%s,%s);  x_test, y_test: (%s,%s); Xs, ys: (%s,%s) " % (x_train.shape, y_train.shape, x_test.shape, y_test.shape, Xs.shape, ys))



Samples length 7480
Targets length 7480
Samples length 2872
Targets length 2872
Samples length 13224
Targets length 0
Data x_train, y_train: ((7480, 48, 12),(7480, 48));  x_test, y_test: ((2872, 48, 12),(2872, 48)); Xs, ys: ((13224, 48, 12),None) 


In [6]:
    Train = True
    Predict = True
    plot = False

In [7]:
    batch_size = 359  # Batch size
    shift = batch_size
    #if Train == False: batch_size = 1

    sequence_length = timesteps  # Number of steps
    learning_rate = 4*10e-5  # 0.0001
    epochs = 1000
    ann_hidden = 16

    n_channels = x_train.shape[2]

    lstm_size = 48  # Number LSTM units
    num_layers = 3  # 2  # Number of layers
    alpha = 0 # regularization coef

In [8]:
    X = tf.placeholder(tf.float32, [None, sequence_length, n_channels], name='inputs')
    Y = tf.placeholder(tf.float32, [None, sequence_length], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    learning_rate_ = tf.placeholder(tf.float32, name='learning_rate')
    is_train = tf.placeholder(dtype=tf.bool, shape=None, name="is_train")

    conv_last_layer = X

    shape = conv_last_layer.get_shape().as_list()
    print('My Conv Shape:',shape)
    CNN_flat = tf.reshape(conv_last_layer, [-1, shape[1] * shape[2]])

    dence_layer_1 = dense_layer(CNN_flat, size=sequence_length * n_channels, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_1")
    lstm_input = tf.reshape(dence_layer_1, [-1, sequence_length, n_channels])

    cell = get_RNNCell(['LSTM'] * num_layers, keep_prob=keep_prob, state_size=lstm_size)
    init_states = cell.zero_state(batch_size, tf.float32)
    
    # For each layer, get the initial state. states will be a tuple of LSTMStateTuples.
    states = get_state_variables(batch_size, cell)

    # Unroll the LSTM
    rnn_output, new_states = tf.nn.dynamic_rnn(cell, lstm_input, dtype=tf.float32, initial_state=states)
    
    # Add an operation to update the train states with the last state tensors.
    update_op = get_state_update_op(states, new_states)
    reset_op = get_state_update_op(states, init_states)
    
    stacked_rnn_output = tf.reshape(rnn_output, [-1, lstm_size])  # change the form into a tensor

    dence_layer_2 = dense_layer(stacked_rnn_output, size=ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_2")
    
    dence_layer_3 = dense_layer(dence_layer_2, size=ann_hidden, activation_fn=tf.nn.relu, batch_norm=False,
                                phase=is_train, drop_out=True, keep_prob=keep_prob,
                                scope="fc_2_2")

    output = dense_layer(dence_layer_3, size=1, activation_fn=None, batch_norm=False, phase=is_train, drop_out=False,
                         keep_prob=keep_prob,
                         scope="fc_3_output")

    prediction = tf.reshape(output, [-1])
    y_flat = tf.reshape(Y, [-1])

    h = prediction - y_flat
    
    tv = tf.trainable_variables()
    regularization_cost = tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv ])

    cost_function = tf.reduce_sum(tf.square(h)) + alpha*regularization_cost
    RMSE = tf.sqrt(tf.reduce_mean(tf.square(h)))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost_function)

    saver = tf.train.Saver()
    
    training_generator = batch_generator(x_train, y_train, batch_size, sequence_length, online=True, online_shift=shift)
    testing_generator = batch_generator(x_test, y_test, batch_size, sequence_length, online=True, online_shift=shift)
    #print(len(list(training_generator)))

    if Train: model_summary(learning_rate=learning_rate, batch_size=batch_size, lstm_layers=num_layers,
                            lstm_layer_size=lstm_size, fc_layer_size=ann_hidden, sequence_length=sequence_length,
                            n_channels=n_channels, path_checkpoint=path_checkpoint, spacial_note='')
        


My Conv Shape: [None, 48, 12]


In [9]:
    with tf.Session() as session:
        tf.global_variables_initializer().run()

        if Train == True:
            saver.restore(session, path_checkpoint)
            print("Model restored from file: %s" % path_checkpoint)

            cost = []
            plot_x = []
            plot_y1 = []
            plot_y2 = []
            iter_train = int(x_train.shape[0]/shift)
            iter_test = int(x_test.shape[0]/shift)
            print("Training set MSE")
            print("No epoches: ", epochs, "No itr: ", iter_train)
            __start = time.time()
            for ep in range(epochs):
                session.run(reset_op)
                
                h1 = []
                t1 = []
                   
                for itr in range(iter_train):
                    ## training ##
                    batch_x, batch_y = next(training_generator)
                    #print("states:",session.run(states))
                    session.run([optimizer, update_op],
                                feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.7, learning_rate_: learning_rate})
                    h_i = h.eval(feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0, learning_rate_: learning_rate})
                    cost.append(np.square(h_i))
                    h1.append(h_i)

                rmse_train = np.sqrt(np.mean(np.square(h1)))
                
                y_pred = []
                for itr in range(iter_test):
                    x_test_batch, y_test_batch = next(testing_generator)
                    #print("states:",session.run(states))
                    h_i, u = session.run([h, update_op], feed_dict={X: x_test_batch, Y: y_test_batch, keep_prob: 1.0, learning_rate_: learning_rate})
                    #session.run(update_op)
                    t1.append(h_i)
                
                rmse_test = np.sqrt(np.mean(np.square(t1)))
                
                #rmse_train = session.run(RMSE, feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
                rmse_test = session.run(RMSE, feed_dict={X: x_test_batch, Y: y_test_batch, keep_prob: 1.0})
                plot_x.append(ep)
                plot_y1.append(rmse_train)
                plot_y2.append(rmse_test)
                
                #print(ep)

                time_per_ep = (time.time() - __start)
                time_remaining = ((epochs - ep) * time_per_ep) / 3600
                print("LSTM", "epoch:", ep, "RMSE-train:", rmse_train, "RMSE-test", rmse_test, "lr", learning_rate,
                      "\ttime/epoch:", round(time_per_ep, 2), "\ttime_remaining: ",
                      int(time_remaining), " hr:", round((time_remaining % 1) * 60, 1), " min", "\ttime_stamp: ",
                      datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S"))
                __start = time.time()

                if ep % 100 == 0 and ep != 0:
                    save_path = saver.save(session, path_checkpoint)
                    if os.path.exists(path_checkpoint + '.meta'):
                        print("Model saved to file: %s" % path_checkpoint)
                    else:
                        print("NOT SAVED!!!", path_checkpoint)

                if ep % 50 == 0 and ep != 0: 
                    plt.plot(plot_x, plot_y1, 'bo', plot_x, plot_y2, 'go')
                    plt.show()
                        
                if ep % 400 == 0 and ep != 0: 
                    learning_rate = learning_rate / 2


            save_path = saver.save(session, path_checkpoint)
            if os.path.exists(path_checkpoint + '.meta'):
                print("Model saved to file: %s" % path_checkpoint)
            else:
                print("NOT SAVED!!!", path_checkpoint)
            plt.plot(plot_x, plot_y1, 'bo', plot_x, plot_y2, 'go')
            plt.show()
        else:
            saver.restore(session, path_checkpoint)
            print("Model restored from file: %s" % path_checkpoint)
            if Predict == True:
                print("Prediction for submit...")
                x_predict = Xs
                y_predict = np.zeros((Xs.shape[0],Xs.shape[1]))

                predict_generator = batch_generator(x_predict, y_predict, batch_size, sequence_length,
                                                       online=True, online_shift=shift)

                full_prediction = []

                iteration = int(x_predict.shape[0] / shift)
                #print("iteration: %i, predgen %s" % (iteration, predict_generator))
                print("#of validation points:", x_predict.shape[0], "#datapoints covers from minibatch:",
                      batch_size * sequence_length, "iterations/epoch", iteration)

                for itr in range(iteration):
                    x_validate_batch, y_validate_batch = next(predict_generator)
                    #print (itr)
                    __y_pred, u = session.run([output, update_op], feed_dict={X: x_validate_batch, Y: y_validate_batch, keep_prob: 1.0})
                    #session.run(update_op)
                    for i in range(batch_size):
                        full_prediction.append(__y_pred[i*sequence_length])
                    #print(__y_pred.shape)
                    
                full_prediction = np.array(full_prediction)
                full_prediction = full_prediction.ravel()
            


INFO:tensorflow:Restoring parameters from ./save/save_lstm/lstm_2_layers
Training set MSE
No epoches:  1000 No itr:  20
LSTM epoch: 0 RMSE-train: 0.63644594 RMSE-test 0.57658833 lr 0.0004 	time/epoch: 4.24 	time_remaining:  1  hr: 10.7  min 	time_stamp:  2018.11.15-18:45:00
LSTM epoch: 1 RMSE-train: 0.61716336 RMSE-test 0.5729039 lr 0.0004 	time/epoch: 3.15 	time_remaining:  0  hr: 52.4  min 	time_stamp:  2018.11.15-18:45:04
LSTM epoch: 2 RMSE-train: 0.61926615 RMSE-test 0.5823159 lr 0.0004 	time/epoch: 3.14 	time_remaining:  0  hr: 52.3  min 	time_stamp:  2018.11.15-18:45:07
LSTM epoch: 3 RMSE-train: 0.6158336 RMSE-test 0.57483804 lr 0.0004 	time/epoch: 3.13 	time_remaining:  0  hr: 52.0  min 	time_stamp:  2018.11.15-18:45:10
LSTM epoch: 4 RMSE-train: 0.620839 RMSE-test 0.5832534 lr 0.0004 	time/epoch: 3.15 	time_remaining:  0  hr: 52.2  min 	time_stamp:  2018.11.15-18:45:13
LSTM epoch: 5 RMSE-train: 0.60860646 RMSE-test 0.5821167 lr 0.0004 	time/epoch: 3.14 	time_remaining:  0  hr: 5

KeyboardInterrupt: 

In [ ]:
x_validate_batch.shape

In [ ]:
full_prediction

In [ ]:
full_prediction.shape

In [ ]:
shft = x_train.shape[0]+x_test.shape[0]+sequence_length

In [ ]:
mean_y

In [ ]:
len(submit_X.index[shft:])

In [ ]:
y_submit = full_prediction*std_y+mean_y
sub_file = pd.DataFrame()
sub_file["timestamp"] = submit_X.index[-2872:]
sub_file["target"] = y_submit[-2872:]
sub_file.to_csv('submit2.csv', index=False)

In [ ]:
# LB 0.4705